In [1]:
from langchain_community.llms import Ollama
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
import os

In [10]:
os.environ["OPENAI_API_KEY"] = "NA"

ollama_model = Ollama(model = "llama3:70b-instruct")

In [11]:
search_tool = SerperDevTool()

In [13]:
# Creating a senior researcher agent with memory and verbose mode
researcher = Agent(
  role='Senior Researcher',
  goal='Uncover groundbreaking technologies in {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "Driven by curiosity, you're at the forefront of"
    "innovation, eager to explore and share knowledge that could change"
    "the world."
  ),
  tools=[search_tool],
  allow_delegation=True,
  llm=ollama_model
)

# Creating a writer agent with custom tools and delegation capability
writer = Agent(
  role='Writer',
  goal='Narrate compelling tech stories about {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "With a flair for simplifying complex topics, you craft"
    "engaging narratives that captivate and educate, bringing new"
    "discoveries to light in an accessible manner."
  ),
  tools=[search_tool],
  allow_delegation=False,
  llm=ollama_model
)

In [14]:
# Research task
research_task = Task(
  description=(
    "Identify the next big trend in {topic}."
    "Focus on identifying pros and cons and the overall narrative."
    "Your final report should clearly articulate the key points,"
    "its market opportunities, and potential risks."
  ),
  expected_output='A comprehensive 3 paragraphs long report on the latest AI trends.',
  tools=[search_tool],
  agent=researcher,
)

# Writing task with language model configuration
write_task = Task(
  description=(
    "Compose an insightful article on {topic}."
    "Focus on the latest trends and how it's impacting the industry."
    "This article should be easy to understand, engaging, and positive."
  ),
  expected_output='A 4 paragraph article on {topic} advancements formatted as markdown.',
  tools=[search_tool],
  agent=writer,
  async_execution=False,
  output_file='output/new-blog-post.md'  # Example of output customization
)

In [7]:
# Forming the tech-focused crew with some enhanced configurations
crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, write_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
#   memory=True,
#   cache=True,
  max_rpm=100,
  share_crew=True
)

In [15]:
# Starting the task execution process with enhanced feedback
result = crew.kickoff(inputs={'topic': 'AI in healthcare'})
print(result)



> Entering new CrewAgentExecutor chain...
Thought: I need to gather information about the latest AI trends in healthcare to identify the next big trend.

Action: Search the internet

Action Input: {"search_query": "latest AI trends in healthcare 


Search results: Title: Five trends converging for AI-enabled healthcare: Why AI and ... - PwC
Link: https://www.pwc.com/gx/en/industries/healthcare/publications/ai-robotics-new-health/five-trends.html
Snippet: The next decade is moving towards medical solutions – using AI, robotics, and virtual and augmented reality – to deliver intelligent solutions for both evidence ...
---
Title: CSIRO report highlights 'extraordinary era' of AI in healthcare
Link: https://www.csiro.au/en/news/all/news/2024/march/ai-trends-for-healthcare-csiro-report-highlights-extraordinary-era-of-ai-in-healthcare
Snippet: The AI Trends for Healthcare report identifies the opportunities and challenges facing the continued and inevitable integration of AI in ...
---
Tit